In [16]:
%load_ext watermark
%watermark -a Filippo_Valle -p pandas,numpy,graph_tool,cloudpickle -m -v -g

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Filippo_Valle

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

pandas     : 1.1.2
numpy      : 1.19.1
graph_tool : 2.43 (commit 5778eb10, )
cloudpickle: 1.6.0

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.11.0-31-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

Git hash: f9cba0c099756c3f5e3ff57c4c8a908d2990f679



In [ ]:
import graph_tool.all as gt
import pandas as pd
import numpy as np
import cloudpickle as pickle
import sys
sys.path.append("../trisbm")
from trisbm import trisbm
import logging
logger = logging.getLogger("cnv_linked")
hdl = logging.StreamHandler()
hdl.setFormatter(logging.Formatter("%(asctime)s - %(message)s"))
logger.addHandler(hdl)
logger.setLevel(logging.DEBUG)

In [ ]:
with open("brca/tetrasbm/trisbm/trisbm.pkl", "rb") as file:
    model = pickle.load(file)

In [ ]:
vertexmap = {}
for i,node in enumerate(np.concatenate((model.documents, model.words, *model.keywords))):
    vertexmap[node]=i

In [ ]:
def get_regulatory_edge():
    for gene in model.words:
        for gene_cnv in model.keywords[1]:
            if gene == gene_cnv[2:]:
                yield (vertexmap[gene], vertexmap[gene_cnv], 100.)
regulatory_edges = get_regulatory_edge()

In [ ]:
print(model.g)
model.g.save("tmp.xml.gz") #need to be stored to remove edges later

In [ ]:
model.g.add_edge_list(regulatory_edges, eprops=[model.g.ep["count"]])

In [ ]:
print(model.g)

In [ ]:
states = []

for level in model.state.levels:
    state = level.copy(g=model.g, bs=model.state.get_bs() + [np.zeros(1)] * 4 , sampling = True)

    S1 = state.entropy()

    for i in range(25):
        logger.debug(i)
        state.multiflip_mcmc_sweep(niter=50, beta=np.inf, verbose=True)
    
    print(model.get_mdl(), S1, state.entropy())
    
    states.append(state)

In [ ]:
model.state = gt.NestedBlockState(g = model.g, bs = [state.b for state in states])
model.g = gt.load_graph("tmp.xml.gz")
model.groups = {} #clean cache

In [ ]:
model.state.levels

In [ ]:
import os

In [ ]:
os.chdir("brca")
os.system("rm -rf cnv_linked && mkdir -p cnv_linked")
os.chdir("cnv_linked")
os.system("mkdir -p trisbm")
os.chdir("trisbm")
model.save_data()
model.dump_model()
os.chdir("../../../")

In [ ]:
pd.read_csv("brca/cnv_linked/trisbm/trisbm_level_1_clusters.csv")

In [ ]:
pd.read_csv("brca/tetrasbm/trisbm/trisbm_level_1_clusters.csv")